In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn.metrics import auc, roc_curve, roc_auc_score, mean_squared_error

In [2]:
df = pd.read_csv('/Users/siak/Downloads/house-prices-advanced-regression-techniques/train.csv')

In [3]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [18]:
categs = list(df.dtypes[df.dtypes == 'object'].index)
df[categs] = df[categs].fillna('NaN')
numers = [f for f in df if f not in (categs + ['Id', 'SalePrice'])]

dummies = pd.get_dummies(df[categs], columns=categs)
dummy_cols = list(set(dummies))
dummies = dummies[dummy_cols]

X = pd.concat([df[numers].fillna(-999), dummies], axis=1)
y = df['SalePrice']

In [9]:
rgr = rfr(n_estimators=10, max_depth=4, n_jobs=-1)

In [12]:
def show_auc(y, y_pred, plot_label='', prin=True):
    fpr, tpr, _ = roc_curve(y, y_pred)
    auc_val = auc(fpr, tpr)
    if prin:
        print('ROC AUC: {0:.4f}'.format(auc_val))
    if plot_label:
        plt.plot(fpr, tpr, label=plot_label)
        plt.xlabel('FPR')
        plt.ylabel('TPR')
    return auc_val

In [36]:
from sklearn.model_selection import StratifiedKFold as skf
spltr = skf(n_splits=10, random_state=42)

aucs = []
for train_ind, test_ind in spltr.split(df, df.SalePrice):
    rgr.fit(X.iloc[train_ind], y.iloc[train_ind])
    y_pred_test = rgr.predict(X.iloc[test_ind])
    y_pred_train = rgr.predict(X.iloc[train_ind])
    print('Train:')
    print(mean_squared_error(y.iloc[train_ind], y_pred_train))
    print('Test:')
    print(mean_squared_error(y.iloc[test_ind], y_pred_test))
    print(' ')
    #show_auc(y.iloc[train_ind], y_pred_train, 'train_10fold_rf')
    #show_auc(y.iloc[test_ind], y_pred_test, 'test_10fold_rf')
#plt.show()
    

/Users/siak/Library/Python/3.7/lib/python/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


Train:
463058128.345076
Test:
3599379852.166895
 
Train:
920778842.6399453
Test:
774470582.1561887
 
Train:
976287892.0381409
Test:
746976865.0390027
 
Train:
928176095.3328708
Test:
782377408.2184933
 
Train:
949377689.01039
Test:
750851443.1883008
 
Train:
944963517.5524249
Test:
680179888.3322896
 
Train:
910406601.6416796
Test:
287044856.7980727
 
Train:
891178365.0243393
Test:
4628567891.051612
 
Train:
875880664.8462111
Test:
1383346326.5177536
 
Train:
879472286.7579252
Test:
494455953.7819784
 


In [48]:
rgr = rfr(n_estimators=10, max_depth=4, n_jobs=-1)
rgr.fit(X, y)
imp = pd.Series(rgr.feature_importances_)
imp.sort_values(ascending=False)

3      0.699679
15     0.113331
13     0.046798
11     0.039948
25     0.035633
         ...   
201    0.000000
202    0.000000
203    0.000000
204    0.000000
0      0.000000
Length: 304, dtype: float64